In [178]:
from splink import Linker, DuckDBAPI
import pandas as pd

db_api = DuckDBAPI()

In [179]:
locs_df = pd.read_csv("production_locations_ds.csv")
locs_df = locs_df.drop(columns=[
    "created_at",
    "updated_at",
    "potential_match_score",
    "potential_match_os_id",
    "found_index"
])
sample_df = locs_df.sample(10000)
sample_df = sample_df[['os_id','name', 'address', 'country_code', 'geocoded_location_type', 'lat', 'lng']]
sample_df

,os_id,name,address,country_code,geocoded_location_type,lat,lng
959228,US2024245M1JVFC,FAIRMONT BOOSTER PUMP STATION,"4150 FAIRMONT BLVD, YORBA LINDA, CA, 92886",US,NaN,33.901764,-117.784844
440927,US2024323GQA1W6,LEWIS MACHINERY SALES LLC,"81-1090 St Johns Rd, Kealakekua, HI, 96750-8120",US,ROOFTOP,19.504870,-155.914198
1094563,CN2023347J14AAH,"Karamay Qinghui Catering Management Co., Ltd.","51-117 Xingye Road, Karamay District, Karamay ...",CN,GEOMETRIC_CENTER,45.570790,84.908370
169322,US2024292D75EZ3,BIG COUNTRY GRAPHICS INC.,"7355 SW 38th St Ste 101, Ocala, FL, 34474-6493",US,ROOFTOP,29.150648,-82.240771
957371,US202424539X82E,COMMERCIAL AUTO & DIESEL ELECTRIC,"257 Garden Hwy, YUBA CITY, CA 95991",US,ROOFTOP,39.126399,-121.609140
...,...,...,...,...,...,...,...
54386,BD2024280J30JTS,Kaderiya Steel (M/S),"Chawk Bazar, Chattogram, Chattogram",BD,APPROXIMATE,22.351774,91.833052
126021,US20242867TFS2B,"Exsilent Usa, Llc","11485 Valley View Rd Eden Prairie, Minnesota 5...",US,ROOFTOP,44.867799,-93.423489
748503,KR2023064CH01V7,Instinctus Factory,"F810~F811, Centumbiz, Jojeong-daero 45",KR,ROOFTOP,37.550814,127.184133
567286,KZ2025053DTSZJ7,BAKELOVE LLP,"Radostovtsa Street, Building 333A, ALMATY CITY...",KZ,APPROXIMATE,43.218914,76.897185


In [180]:

test_rec = locs_df.sample(1)[['os_id','name', 'address', 'country_code', 'geocoded_location_type', 'lat', 'lng']]
test_rec

,os_id,name,address,country_code,geocoded_location_type,lat,lng
409823,US20243132BG9QM,DICKINSON TIRE AUTO INC,"1117 St Rt 11B, BRUSHTON, NY, 12916",US,ROOFTOP,44.76946,-74.520423


In [181]:
import json

with open("record_linkage_model_beta.json", "r") as file:
    record_linkage_model = json.load(file)

linker = Linker(
    sample_df,
    settings=record_linkage_model,
    db_api=db_api,
)

In [182]:
df_predictions = linker.inference.predict(threshold_match_probability=0.2)
df_predictions.as_pandas_dataframe()

Blocking time: 1.35 seconds
Predict time: 6.41 seconds


,match_weight,match_probability,os_id_l,os_id_r,name_l,name_r,gamma_name,bf_name,address_l,address_r,...,bf_geocoded_location_type,bf_tf_adj_geocoded_location_type,lng_l,lng_r,lat_l,lat_r,gamma_lat_lng,bf_lat_lng,country_code_l,country_code_r
0,-0.765581,0.370363,IN2024178X2E668,IN2024183Z9PQSS,Saket Warehouse,Mahadev Warehouse,4,19.164529,"Village Ankhmau, Hoshangabad, MADHYA PRADESH","Village Hathna, Jabalpur, MADHYA PRADESH",...,1.447005,2.368855,77.965299,79.950048,22.696293,23.235092,-1,1.000000,IN,IN
1,-1.769118,0.226837,CN2024009NSQNA6,CN2024009RFFXNA,"Pishan County Baihe Tiya Clothing Co., Ltd.","Pishan County Style Clothing Co., Ltd",5,295.757994,"North Qingquan Community, Bashibulake Village,...","Group 4, Kumuyigele Village, Sangzhu Town, Pis...",...,1.447005,2.368855,78.528223,78.460758,37.488660,37.184338,-1,1.000000,CN,CN
2,-1.769118,0.226837,CN2023347Q4D45Q,CN2023347Y738XK,"Yili Dingsu Trading Co., Ltd.","Yili Honglian Trading Co., Ltd.",5,295.757994,3555 Ningyuan County C Complex Building 101-9 ...,"225 Wangjinghuating 1 Building, Wangjing Sunsh...",...,1.447005,2.368855,81.527320,81.277250,43.977150,43.909500,-1,1.000000,CN,CN
3,-0.066310,0.488511,CN2020191ZT3297,CN2023060MJCW1D,"Jiangyin Dayao Knitting Clothing Co., Ltd.","Nantong Century Texhong Textile Co., Ltd.",4,19.164529,"NO. 227,Chenglu Road, Huashi Town, Jiangyin Ci...","No. 2 Jianghai West Road, Shuangdian Town, Rud...",...,1.447005,3.846273,120.447878,121.166210,31.842869,32.311020,-1,1.000000,CN,CN
4,-1.200964,0.303128,CN2022249YMWADB,CN2023066WCK3RT,"Suzhou Kangjian Textile Co., Ltd.","Suzhou Ouyifang Textile Co.,Ltd.",6,270.063855,"Room 1602, Building 7, Jincheng Commercial Cen...","No. 1538, South Second Ring Road, Shengze Town...",...,1.447005,3.846273,120.672370,120.642270,30.901450,30.878890,-1,1.000000,CN,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,-0.765581,0.370363,KZ2025056FPAJ2W,KZ2025056P03WP4,ASYL-MEKEN KM AGRICULTURAL PRODUCTION COOPERATIVE,AGRO ASYLDANDYRU AGRICULTURAL PRODUCTION COOPE...,4,19.164529,"Tuştşykol Street, Building 4, AKMOLA REGION, Q...","A.Ziiaeva Street, Building 44, TURKISTAN REGIO...",...,1.447005,2.368855,70.050000,69.812089,50.766667,42.472664,-1,1.000000,KZ,KZ
847,-1.303278,0.288360,US2024301RKHZZE,US20243121FSFWA,HARRIS WOODWORKS LLC,"SUPERIOR ELECTRIC MOTOR SALES & SERVICE, INC.",3,1.684434,"1301 2ND ST, MARQUETTE, MI 49855-3005","1740 PRESQUE ISLE AVE, MARQUETTE, MI, 49855-2131",...,1.447005,0.752618,-87.393836,-87.395834,46.555725,46.561767,2,24.668661,US,US
848,-1.303278,0.288360,US202431893DJ29,US2024318XZH85P,"REAL GEMS, INC.",CRESCENT WEDDING RINGS INC,3,1.684434,"36 West 44th Street, Ste 816, New York, NY, 10036","36 W 47th St Ste 306, New York, NY, 10036-8637",...,1.447005,0.752618,-73.982023,-73.980421,40.755236,40.757196,2,24.668661,US,US
849,0.632242,0.607839,US202501459CJXZ,US2025014X5B18C,GRACE CASUALS INC,"JCA APPAREL, INC.",4,19.164529,"1509 W PICO BLVD, LOS ANGELES, CA 90015","1753 E OLYMPIC BLVD, LOS ANGELES, CA 90021",...,1.447005,0.752618,-118.275434,-118.239644,34.044281,34.028566,1,8.293747,US,US


In [183]:
lk_df = df_predictions.as_pandas_dataframe().sort_values("match_weight", ascending=False)

In [184]:
lk_df[lk_df["match_probability"] > 0.8].to_csv("lk_df.csv", index=False)

In [185]:
df_inc = linker.inference.find_matches_to_new_records(
    sample_df.sample(10), blocking_rules=[]
).as_pandas_dataframe()
df_inc.sort_values("match_weight", ascending=False)

,match_weight,match_probability,os_id_l,os_id_r,name_l,name_r,gamma_name,bf_name,address_l,address_r,...,tf_geocoded_location_type_l,tf_geocoded_location_type_r,bf_geocoded_location_type,bf_tf_adj_geocoded_location_type,lng_l,lng_r,lat_l,lat_r,gamma_lat_lng,bf_lat_lng
0,11.028650,0.999522,US2024299H9S13V,US2024299H9S13V,"VALLEY SIGN & BANNER CO., INC.","VALLEY SIGN & BANNER CO., INC.",8,184.212176,"243 CREST LAKE DRIVE, BIRMINGHAM, AL, 35244","243 CREST LAKE DRIVE, BIRMINGHAM, AL, 35244",...,0.634571,0.634571,1.447005,0.752618,-86.773374,-86.773374,33.362379,33.362379,5,26.420274
1,11.028650,0.999522,US20242927T3SNF,US20242927T3SNF,D&T DISTRIBUTION INC.,D&T DISTRIBUTION INC.,8,184.212176,"5130 COMMERCIAL DRIVE SUITE H, MELBOURNE, FL, ...","5130 COMMERCIAL DRIVE SUITE H, MELBOURNE, FL, ...",...,0.634571,0.634571,1.447005,0.752618,-80.667033,-80.667033,28.191896,28.191896,5,26.420274
3,11.028650,0.999522,US2024284GSB4YM,US2024284GSB4YM,Uva Imports Llc,Uva Imports Llc,8,184.212176,"5555 Oakbrook Pkwy Ste 535 Norcross, Georgia 3...","5555 Oakbrook Pkwy Ste 535 Norcross, Georgia 3...",...,0.634571,0.634571,1.447005,0.752618,-84.184588,-84.184588,33.920372,33.920372,5,26.420274
4,11.028650,0.999522,US2024309G8ZH26,US2024309G8ZH26,AJ DELUCA CONTRACTORS INC.,AJ DELUCA CONTRACTORS INC.,8,184.212176,"401 W Old Liberty Rd, Sykesville, MD, 21784-9032","401 W Old Liberty Rd, Sykesville, MD, 21784-9032",...,0.634571,0.634571,1.447005,0.752618,-77.010249,-77.010249,39.428166,39.428166,5,26.420274
6,11.028650,0.999522,US2024303SWXQ10,US2024303SWXQ10,"GARNER CONSTRUCTION CONSULTING, LLC","GARNER CONSTRUCTION CONSULTING, LLC",8,184.212176,"4868 Countrywood Mnr, Tupelo, MS 38801-7964","4868 Countrywood Mnr, Tupelo, MS 38801-7964",...,0.634571,0.634571,1.447005,0.752618,-88.787713,-88.787713,34.278999,34.278999,5,26.420274
8,11.028650,0.999522,US20243063NS3YB,US20243063NS3YB,"BOUDREAUX'S NEW DRUG STORE, LLC","BOUDREAUX'S NEW DRUG STORE, LLC",8,184.212176,"404 E. PRIEN LAKE RD, LAKE CHARLES, LA, 70601-...","404 E. PRIEN LAKE RD, LAKE CHARLES, LA, 70601-...",...,0.634571,0.634571,1.447005,0.752618,-93.216344,-93.216344,30.197903,30.197903,5,26.420274
9,8.658548,0.997531,CN2023346WEYBE6,CN2023346WEYBE6,"Urumqi Evergrande Guanghua Tempered Glass Co.,...","Urumqi Evergrande Guanghua Tempered Glass Co.,...",8,184.212176,"1011 South Shuguang Road, Midong District, Uru...","1011 South Shuguang Road, Midong District, Uru...",...,0.124169,0.124169,1.447005,3.846273,87.768450,87.768450,43.969576,43.969576,-1,1.000000
2,7.959277,0.995998,CN20243003SWHDJ,CN20243003SWHDJ,"Guangdong Hengsheng Textile Co., Ltd.","Guangdong Hengsheng Textile Co., Ltd.",8,184.212176,"81 Yinlang North Road, Zhushan Community, Dala...","81 Yinlang North Road, Zhushan Community, Dala...",...,0.201612,0.201612,1.447005,2.368855,113.943978,113.943978,22.940016,22.940016,-1,1.000000
5,7.959277,0.995998,TR2020191413WJB,TR2020191413WJB,ABC TEKSTIL SANAYI VE TICARET A.S.,ABC TEKSTIL SANAYI VE TICARET A.S.,8,184.212176,Pinarkent Mahallesi Sahinler Kume Eveler No :1...,Pinarkent Mahallesi Sahinler Kume Eveler No :1...,...,0.201612,0.201612,1.447005,2.368855,29.118710,29.118710,37.806691,37.806691,-1,1.000000
7,7.959277,0.995998,VN20212246FG5YS,VN20212246FG5YS,Delta Global Sourcing Ltd.,Delta Global Sourcing Ltd.,8,184.212176,"No. 5 Thong Nhat Avenue, Song Than 2 Ind. Zone...","No. 5 Thong Nhat Avenue, Song Than 2 Ind. Zone...",...,0.201612,0.201612,1.447005,2.368855,106.752742,106.752742,10.896476,10.896476,-1,1.000000
